<h1>Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Words-in-the-filing" data-toc-modified-id="Words-in-the-filing-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Words in the filing</a></span></li><li><span><a href="#Pages-in-the-filing" data-toc-modified-id="Pages-in-the-filing"><span class="toc-item-num">2&nbsp;&nbsp;</span>Pages in the filing</a></span></li><li><span><a href="#Count-images" data-toc-modified-id="Count-images-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Count images</a></span></li><li><span><a href="#Find-the-sections" data-toc-modified-id="Find-the-sections-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Find the sections</a></span></li><li><span><a href="#Top-5-Salaries" data-toc-modified-id="Top-5-Salaries-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Top 5 Salaries</a></span></li><li><span><a href="#Extracting-information" data-toc-modified-id="Extracting-information-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Extracting information</a></span></li></ul></div>

In [ ]:
import re
import os
from collections import Counter
from IPython.display import HTML

Using the filing at `cik=1652044`, `accession=0001308179-17-000170` ([link](https://www.sec.gov/Archives/edgar/data/1652044/000130817917000170/lgoog2017_def14a.htm)), read in the HTML and answer the following questions:



  1. How many words are in the filing?
  1. How many pages are in the filing?
  1. How many images are included in the filing?
  1. What are the different sections of the proxy statement? (hint: see the table of contents we found above).
  1. What are the top 5 people (by salary) paid at Google?
  1. *WITHOUT CODING*: Describe how you would go about extracting this information programatically.
     1. What format is this information in?
     1. Do you think it's repeatable for not-Google?
     1. Would you use HTML or plain text to get this information?
     1. Extra credit: Write out some [pseudo-code](https://www.vikingcodeschool.com/software-engineering-basics/what-is-pseudo-coding) for your approach, or just the steps in plain english.

In [ ]:
# Import the filing
import html2text
from bs4 import BeautifulSoup
from pyedgar import Filing

# Load the filing
filing = Filing(1652044, '0001308179-17-000170')

# Get the HTML copy
html = filing.documents[0]['full_text']

# Get the beautifulsoup object of the HTML
soup = BeautifulSoup(html, 'lxml')

# Get the plaintext version of the html
h = html2text.HTML2Text()
h.ignore_links = False
h.ignore_tables = False
plaintext = h.handle(html)

# Words in the filing

In [ ]:
# Using the word finder from last homework
re_word_finder = re.compile(
    r'\b'                 # word-boundry, so match beginning or ends of words
    r"(?:[^\W\d_]|[-'])+" # (?: is a non-capture group, just means we can use | for or.
                          # ^\W\d_ means not (^) a non-word, or a digit or underscore
    r'\b'                 # ending word-boundry
    , re.IGNORECASE
)

In [ ]:
# Counting words in the plaintext
num_words = len(re_word_finder.findall(plaintext))
print(f"Length of text: {len(plaintext):,d}")
print(f"Number of words: {num_words:,d}")

In [ ]:
# Counting words in the plaintext
num_words = len(re_word_finder.findall(soup.get_text()))
print(f"Length of text: {len(soup.get_text()):,d}")
print(f"Number of words: {num_words:,d}")

That's odd, I would think that plaintext (which leaves more formatting in) would be longer.
Let's take a look at why.

In [ ]:
ex_sec = plaintext.index("Post-Employment and Change in Control Payments")
print(plaintext[ex_sec: ex_sec+2000])

In [ ]:
ex_sec = soup.get_text().index("Post-Employment and Change in Control Payments")
print(soup.get_text()[ex_sec: ex_sec+2000])

Let's do a wordcount to see the top words

In [ ]:
num_pt = Counter(map(lambda x: x.lower(), re_word_finder.findall(plaintext)))
num_pt.most_common(10)

In [ ]:
num_soup = Counter(map(lambda x: x.lower(), re_word_finder.findall(soup.get_text())))
num_soup.most_common(10)

What words are in one and not the other?

In [ ]:
in_pt_not_soup = set(num_pt.keys()) - set(num_soup.keys())
in_pt_not_soup

In [ ]:
in_soup_not_pt = set(num_soup.keys()) - set(num_pt.keys())
in_soup_not_pt

Do those words alone explain the discrepancy? 

In [ ]:
sum(1 for w in re_word_finder.findall(plaintext) if w in in_pt_not_soup)

In [ ]:
sum(1 for w in re_word_finder.findall(soup.get_text()) if w in in_soup_not_pt)

In [ ]:
dif_sum = 0
for k in num_pt.keys():
    dif = num_pt[k] - num_soup[k]
    if dif != 0:
        print(k, num_pt[k], num_soup[k])
    dif_sum += dif
print(dif_sum)

In [ ]:
dif_sum = 0
for k in num_soup.keys():
    dif = num_soup[k] - num_pt[k]
    if dif != 0:
        print(k, num_soup[k], num_pt[k])
    dif_sum += dif
print(dif_sum)

I notice that alphabet seems to be pretty different:

In [ ]:
num_pt['alphabet'], num_soup['alphabet']

Let's look through and see what matches from each text:

In [ ]:
_ = [w for w in re_word_finder.findall(plaintext) if 'alphabet' in w.lower()]
len(_)

In [ ]:
_ = [w for w in re_word_finder.findall(soup.get_text()) if 'alphabet' in w.lower()]
len(_)

If we look through, we see that plaintext gets lots of apostrophies, but soup.get_text() doesn't.

In [ ]:
_txt = soup.get_text()
for match in re_word_finder.finditer(_txt):
    if 'alphabet' not in match.group(0).lower():
        continue
    print(_txt[match.start()-10:match.end()+10].replace('\n', '\t'))

In [ ]:
for match in re_word_finder.finditer(plaintext):
    if 'alphabet' not in match.group(0).lower():
        continue
    print(plaintext[match.start()-10:match.end()+10].replace('\n', '\t'))

In [ ]:
# Using the word finder from last homework
re_word_finder = re.compile(
    r'\b'                  # word-boundry, so match beginning or ends of words
    r"(?:[^\W\d_]|[-'’])+" # Add ’, which is not just single quote, but single end quote
    r'\b'                  # ending word-boundry
    , re.IGNORECASE
)

In [ ]:
# Counting words in the plaintext
num_words = len(re_word_finder.findall(plaintext))
print(f"Length of text: {len(plaintext):,d}")
print(f"Number of words: {num_words:,d}")

In [ ]:
# Counting words in the plaintext
num_words = len(re_word_finder.findall(soup.get_text()))
print(f"Length of text: {len(soup.get_text()):,d}")
print(f"Number of words: {num_words:,d}")

Magnifique.

See, regexes take lots of work, most of it iterative like this.

# Pages in the filing

Most EDGAR filings are broken up into pages. 
Maybe not all, but a lot.

They look like this:

In [ ]:
soup.findAll(style='font: 9pt Arial, Helvetica, Sans-Serif; margin: 0pt 0; text-align: right', limit=2)[-1]

It's just a break between pages, so we can look for those and count them.
Looking at the page numbers will help us verify that we're doing it right.

In [ ]:
# re.DOTALL is needed to let .* match newlines
alpha_to_number = re.compile(r'^\s*alphabet.*[^\d]+\d{1,3}\s*$', re.IGNORECASE | re.DOTALL)

pages = []
for match in soup.findAll('p'):
    txt = match.get_text()
    if not alpha_to_number.search(txt):
        continue
    print(txt.strip().replace('\n', ' '))
    pages.append(txt.strip().split()[-1])

In [ ]:
print(pages)

In [ ]:
print(f"Number of pages: {len(pages)}")

Obviously, we started at 4, so really there's 84 pages.
But close enough.

# Count images

In [ ]:
num_images = len(soup.findAll('img'))
print(f"Number of images: {num_images:,d}")

What are these images?

In [ ]:
soup.findAll('img')

# Find the sections

From `3_Scraping.ipynb`, You might remember we found the table of contents by looking for headers.
Let' try that again.

In [ ]:
soup.findAll(text=re.compile('table.{0,20}contents', re.I))

In [ ]:
toc = soup.findAll(text=re.compile('table.{0,20}contents', re.I))[0]
toc.parent

In [ ]:
toc.parent.parent

In [ ]:
if toc.parent.parent.parent.name == 'body':
    print("We've gone far enough.")

So let's see if the TOC is the first table after that header we found above.

In [ ]:
HTML(toc.parent.parent.findNext('table').prettify())

Because we always look at our data when we develop algorithms, we instantly notice that the CD&A is missing.
Why?

Stupid pages.

In [ ]:
html = toc.parent.parent.findNext('table')
html.findNext('table')

Ug, time to go look at the filing again.

[Here's the link again](https://www.sec.gov/Archives/edgar/data/1652044/000130817917000170/lgoog2017_def14a.htm).

In [ ]:
table1 = toc.parent.parent.findNext('table')
table2 = (table1.findNext(text=re.compile('compensation\s+discussion\s+and\s+analysis', re.I))
                .findParent('table'))
HTML(table1.prettify() + table2.prettify())

How can we be sure that we're done?
That's the hard part.

You might think of looking for tables that just have words and one number, which sequentially increase.
Or you could think of classifying each heading you can think of and then looking for all of them, grabbing the tables that enclose them.

Hopefully you now see that parsing HTML is hard, and requires a lot of iteration on documents to see what is and isn't working.

In [ ]:
i = 1
for tab in [table1, table2]:
    for h1 in tab.findAll(attrs={'style':re.compile('font: 12pt Arial, Helvetica, Sans-Serif;')}):
        text = h1.get_text().strip()
        if len(text) <= 5:
            # These are the page numbers
            continue
        text = re.sub('\s+', ' ', text)
        print("{})".format(i), text)
        i += 1

# Top 5 Salaries

I always start by finding it myself.
For this, the inspector in chrome is absolutely invaluable.
Learn it well, it will be your friend.

[Documentation](https://developers.google.com/web/tools/chrome-devtools/inspect-styles/)

I'm going to start by finding the "Summary Compensation Table", and taking the first table after that with salary in the header.
Well that's my plan, we'll see if it works.

In [ ]:
sum_comp = soup.findAll(text='Summary Compensation Table')
sum_comp

We've been here before, we probably want the second one.
If we're in a table, that's the wrong one (TOC).

In [ ]:
# The carat means the start of the text, so not in the middle of a paragraph
for sum_comp in soup.findAll(text=re.compile('^summary\s+compensation\s*table', re.I)):
    print(sum_comp)
    if sum_comp.findParent('table'):
        print("We're in a table, move on.")
        continue
    
    print("We're not in a table. Go with this one.")
    break

In [ ]:
sum_comp

Now let's find the next tables.

In [ ]:
sum_table = (sum_comp.findNext('table', text=re.compile('salary', re.I))
                     .findParent('table'))

Hmm, I wonder why not.
Let's try some things:

In [ ]:
sum_comp.findNext('table', text=re.compile('salary', re.I))

In [ ]:
sum_comp.findNext('table', text=re.compile('salary', re.I))

Hmm, nothing.
Don't ask why I'd try this, but what about row instead of table?

In [ ]:
sum_comp.findNext('tr', text=re.compile('salary', re.I))

In [ ]:
sum_comp.findNext('td', text=re.compile('salary', re.I))

Okay... odd.
Well this works I guess.

In [ ]:
sum_table = (sum_comp.findNext('td', text=re.compile('salary', re.I))
                     .findParent('table'))

In [ ]:
HTML(sum_table.prettify())

That's the table we want.
Now let's grab all the first columns with names in them.

In [ ]:
for row in sum_table.findAll('tr'):
    cell = row.find('td') # the first cell
    print(cell.get_text())

Hmm, don't like all that noise.
A quick look shows that the names are bolded.
Let's just keep the bold then.

In [ ]:
for row in sum_table.findAll('tr'):
    # The named individuals are in bold.
    bold_name_in_cell = row.find('td').find('b')
    print(bold_name_in_cell.get_text())

Oh, some are none if 'b' isn't found. Ignore those.

In [ ]:
for row in sum_table.findAll('tr'):
    # The named individuals are in bold.
    bold_name_in_cell = row.find('td').find('b')
    if not bold_name_in_cell:
        continue
    print(bold_name_in_cell.get_text())

Lastly, we just want that first name in the cell, not the super-script (6):

In [ ]:
for row in sum_table.findAll('tr'):
    # The named individuals are in bold.
    bold_name_in_cell = row.find('td').find('b')
    if not bold_name_in_cell:
        continue
    print(bold_name_in_cell.contents[0])

So now we want to drop that first one.
We know that these tables will have the hear associated with the salary in them, so let's check for that.

In [ ]:
for row in sum_table.findAll('tr'):
    # The named individuals are in bold.
    if not row.find(text=re.compile('(?:19|20)\d\d')):
        continue
    bold_name_in_cell = row.find('td').find('b')
    if not bold_name_in_cell:
        continue
    print(bold_name_in_cell.contents[0])

This works, but we probably shouldn't be proud of it.
Run it on any other document, and we'd get nothing.

That figuring out a robust algorithm to handle all cases is where the real struggle lies.

# Extracting information

**Explanation of how I would think about approaching this**

  1. Look at a few examples, try to find the similarities between them, as well as the variances.
  1. Search across all Proxy statements for a simple regex of the name of the section or table title.
  1. Based on how many proxies successfully match the regex, rework it until you get good coverage.
  1. Start with a random document in a for loop, break out
     1. e.g. `for row in df.iterrows(): filing=Filing(row.cik, row.accession);break`
  1. Use BS to search for text around the table, then findNext('table') to get the table (if it's tabulated, which we hope it is).
  1. Repeat this for a few more filings, test generalizability.
  1. Probably fix many bugs, but do so iteratively, building up conditionals or a more robust regex as you go.
  1. Extract all the tables possible
  1. Extract data from tables:
     1. Loop through tables and extract all column headers.
     1. Determine similarities between them, group into whatever categories are reasonable.
     1. Loop through rows in table, saving names, amounts by category into a list.
     1. Load it all into a dataframe and save to disk.